In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:

words = open('names.txt','r').read().splitlines()

print(len(words))
print(max(len(w) for w in words))
print(words[:8])

In [ ]:

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

print(itos)
print(vocab_size)



In [ ]:
import random
random.seed(42)
random.shuffle(words)

In [ ]:
block = 3

def build_dataset(words):



  X, Y = [], []
  for w in words:
    context = [0] * block
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]


  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape,Y.shape)

  return X, Y


n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [ ]:
for x,y in zip(Xtr[:20],Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

In [ ]:
# Neural Net


class Linear:
  def __init__(self,fan_in,fan_out,bias=True):
    self.weight = torch.randn((fan_in,fan_out))/fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self,x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


# -------------------------------------------------------------------------------------------

class BatchNorm1d:

  def __init__(self,dim,eps=1e-5,momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # Trainable parameters using backprop
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # Buffers
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self,x):

    if self.training:
      xmean = x.mean(0,keepdim=True)
      xvar = x.var(0,keepdim=True)
    else:
      xmean = self.running_mean
      xvar = self.running_var

    xhat = (x-xmean)/torch.sqrt(xvar+self.eps)
    self.out = self.gamma*xhat + self.beta

    # Updating Buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1-self.momentum)*self.runnning_mean + self.momentum*xmean
        self.running_var = (1-self.momentum)*self.runnning_var + self.momentum*xvar

    return self.out


  def parameters(self):
    return [self.gamma,self.beta]


# -------------------------------------------------------------------------------------------


class Tanh:

  def __call__(self,x):
    self.out = torch.tanh(x)
    return self.out

  def parameters(self):
    return []


# -------------------------------------------------------------------------------------------

class Embedding:

  def __init__(self,num_embeddings,embedding_dim):
    self.weight = torch.randn((num_embeddings,embedding_dim))

  def __call__(self,IX):
    self.out = self.weight[IX]
    return self.out

  def parameters(self):
    return [self.weight]

# -------------------------------------------------------------------------------------------

class Flatten:
  def __call__(self,x):
    self.out = x.view(x.shape[0],-1)
    return self.out

  def parameters(self):
    return []

# -------------------------------------------------------------------------------------------

class Sequential:

  def __init__(self,layers):
    self.layers = layers

  def __call__(self,x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
torch.manual_seed(42);

In [ ]:
n_emb = 10 # dimensionality of character embedding vectors
n_hidden = 200 # no of neurons in hidden layers of MLP

# C = torch.randn((vocab_size,n_emb))
model = Sequential([
    Embedding(vocab_size,n_emb),
    Flatten(),
    Linear(n_emb*block,n_hidden,bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
])

with torch.no_grad():
  model.layers[-1].weight *= 0.1 # scaled down for better initialization of network

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))

for p in parameters:
  p.requires_grad = True

In [ ]:

max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  ix = torch.randint(0,Xtr.shape[0],(batch_size,))
  Xb,Yb = Xtr[ix],Ytr[ix]

  # forward pass
    # emb = C[Xb]
    # x = emb.view(emb.shape[0], -1) # concatenate the vectors
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb)

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update (stochastic gradient descent)
  lr = 0.1 if i<150000 else 0.01
  for p in parameters:
    p.data += -lr*p.grad

  # tracking stats
  if i % 10000 == 0:
    print(f'{i:7d}/{max_steps:7d} : {loss.item():.4f}')

  lossi.append(loss.log10().item())





In [ ]:
plt.plot(torch.tensor(lossi).view(-1,1000).mean(1))

In [ ]:
# put layers into evaluation mode
for layer in model.layers:
  layer.training = False

In [ ]:
# evaluate the loss
@torch.no_grad()
def split_loss(split):
  x,y = {
      'train': (Xtr,Ytr),
      'val' : (Xdev,Ydev),
      'test': (Xte,Yte),
  }[split]

  logits = model(x)
  loss = F.cross_entropy(logits,y)
  print(split,loss.item())

split_loss('train')
split_loss('val')


In [ ]:
# Sampling from the model

for _ in range(20):

  out = []
  context = [0]*block

  while True:
    # emb = C[torch.tensor([context])] # (1,block_size,n_emb)
    # x = emb.view(emb.shape[0], -1) # concatenating vectors
    logits = model(torch.tensor([context]))
    probs = F.softmax(logits,dim = 1)

    ix = torch.multinomial(probs,num_samples=1).item()

    context = context[1:] + [ix]
    out.append(ix)

    if ix == 0:
      break

  print(''.join(itos[i] for i in out))